## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.   

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

## Цель проекта:

Анализ базы данных, содержащей информацию о книгах, издательствах, авторах и пользовательских обзорах книг.

##  Подключение к БД

In [6]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
from IPython.display import Image

In [7]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)


In [8]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [9]:
con=engine.connect()

In [10]:
pd.io.sql.read_sql(sql=text(query), con = con)

NameError: name 'query' is not defined

In [ ]:
# Функция выполнения sql запроса

def func(sql):
    return pd.read_sql(sql,con=engine)

In [ ]:
# ER-Диаграмма
Image(url='https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&table=block&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=2000&userId=&cache=v2')

## Вывод первых строк таблиц

In [ ]:
# для книг
sql_books = '''
SELECT *
FROM books
'''
books = func(sql_books)
books.head()

In [ ]:
# для авторов
sql_authors = '''
SELECT *
FROM authors
'''
authors = func(sql_authors)
authors.head()

In [ ]:
# для рейтингов
sql_ratings = '''
SELECT *
FROM ratings
'''
ratings = func(sql_ratings)
ratings.head()

In [ ]:
# для обзоров
sql_reviews = '''
SELECT *
FROM reviews
'''
reviews = func(sql_reviews)
reviews.head()

In [ ]:
# для издательств
sql_publishers = '''
SELECT *
FROM publishers
'''
publishers = func(sql_publishers)
publishers.head()

## Количество книг, вышедших после 1 января 2000 года

In [ ]:
query = '''
SELECT COUNT(book_id)
FROM books 
WHERE publication_date > '2000-01-01'
'''

func(query)

После первого января 2000 года вышло 819 книг

## Количество обзоров и средняя оценка для каждой книги

In [ ]:
query = '''
SELECT books.book_id as id,
       books.title as title,
       AVG(ratings.rating) as avg_rating,
       COUNT(DISTINCT reviews.review_id) as reviews_count
FROM books
LEFT JOIN reviews on reviews.book_id = books.book_id
LEFT JOIN ratings on ratings.book_id = books.book_id
GROUP BY title, id
ORDER BY reviews_count DESC
'''
func(query)

Для каждой книги показали средний рейтинг и количество обзоров. Лидером по обзорам является книга Twilight (Twilight #1), при этом она обладает не очень высоким рейтингом, всего 3.66.

## Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

In [ ]:
query = '''
SELECT pub.publisher as publisher_name,
       COUNT(DISTINCT books.book_id) as books_count
FROM publishers as pub
 LEFT JOIN books on pub.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publisher_name
ORDER BY books_count DESC
LIMIT 10;
'''
func(query)

Лидером по количеству изданных книг толще 50 страниц является издательство Penguin Books

## Определение автора с самой высокой средней оценкой книг

Здесь будут учитываться только книги с 50 и более оценками

In [ ]:
query = '''
SELECT authors.author,
       ROUND(AVG(ratings.rating),2) AS avg_rating
                   
FROM ratings            
LEFT JOIN books AS books ON ratings.book_id=books.book_id
LEFT JOIN authors AS authors ON books.author_id=authors.author_id
WHERE ratings.book_id IN (SELECT ratings.book_id
                          FROM ratings
                          GROUP BY ratings.book_id
                          HAVING COUNT(ratings.book_id) >= 50)
GROUP BY authors.author
ORDER BY avg_rating DESC
LIMIT 1
'''
func(query)

Автором с самой высокой средней оценкой книг по заданому критирею явлется J.K. Rowling/Mary GrandPré имеет средний рейтинг 4.29

##  Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [ ]:
query = '''
WITH t as
    (SELECT username,
            COUNT(review_id) as review_count
     FROM reviews
     WHERE username in (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating) > 48)
    GROUP BY username)
    
SELECT AVG(review_count)
FROM t
       
'''
func(query)

Среднее количество обзоров пользователей, которые поставили более чем по 48 оценок, равно 24.

## Общий вывод

После первого января 2000 года вышло 819 книг;   
Лидером по обзорам является книга Twilight (Twilight #1), при этом она обладает не очень высоким рейтингом, всего 3.66;   
Лидером по количеству изданных книг толще 50 страниц является издательство Penguin Books;   
Автором с самой высокой средней оценкой книг по заданому критирею явлется J.K. Rowling/Mary GrandPré имеет средний рейтинг 4.29;   
Среднее количество обзоров пользователей, которые поставили более чем по 48 оценок, равно 24.